In [27]:
import pdfplumber
from statistics import mean


def extract_slides(pdf_path: str,
                   min_avg_len: int = 10,
                   max_lines: int = 20,
                   merge_tol: float = 2.0):
    slides = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            txt = page.extract_text() or ""
            lines = [l for l in txt.splitlines() if l.strip()]
            if not lines:                      # page vide
                continue
            if mean(map(len, lines)) < min_avg_len or len(lines) > max_lines:
                continue

            words = page.extract_words(extra_attrs=["size"], use_text_flow=True)
            # regroupe en lignes
            rows, cur, cur_top = [], [], None
            for w in sorted(words, key=lambda w: w["top"]):
                if cur_top is None or abs(w["top"] - cur_top) <= merge_tol:
                    cur.append(w); cur_top = cur_top or w["top"]
                else:
                    rows.append(cur); cur, cur_top = [w], w["top"]
            if cur: rows.append(cur)

            first5 = rows[:5]
            title = (
                " ".join(w["text"] for w in max(first5,
                                                key=lambda r: mean(w["size"] for w in r)))
                if first5 else f"Slide {page.page_number}"
            )

            slides.append({
                "id": f"SL_{page.page_number:03}",   # <─ identifiant slide
                "title": title,
                "content": txt.strip()
            })
    return slides




slides = extract_slides("./volume/slides/cours_4.pdf")

for slide in slides[:10]:
    print(f"Slide number: {slide['id']}")
    print(f"Slide title: {slide['title']}")
    print(slide["content"])
    print("="*100)

Slide number: SL_001
Slide title: Biochimie du Gène et de l'Expression Génique
Biochimie du Gène et de l'Expression Génique
(13 h cours – 11 chapitres)
Drs. Pascal Dollé, Didier Devys, Philippe Kastner
(Institut de Génétique et de Biologie Moléculaire et
Cellulaire [IGBMC], Parc d'innovation, Illkirch)
Dr. Jean Muller (Inserm U 1112, Génétique Médicale,
CRBS, Faculté de Médecine)
Slide number: SL_002
Slide title: 1. Structure et propriétés des acides nucléiques
P.
Dollé
D.
Devys
Sommaire / Plan du cours (1)
1. Structure et propriétés des acides nucléiques
2. Réplication de l'ADN
3. Réparation de l'ADN (anomalies et mécanismes de correction)
4. Recombinaison de l’ADN (définition, généralités)
5. Transcription de l'ADN (synthèse des ARN)
6. Modifications (maturation) de l'ARN chez les eucaryotes
7. Régulation de la transcription
8. Code génétique
9. Traduction (synthèse des protéines) P. Kastner
10. Organisation et évolution des génomes eucaryotes animaux
11. Méthodes en biologie molécul

In [28]:
from models import Slides, Content, ContentSection, SectionSlideMapping

formated_slides = []


for slide in slides:
    #-print(type(slide['id']), type(slide['title']), type(slide['content']))
    formated_slides.append(Slides(id=slide['id'], 
                                title=slide['title'],
                                content=slide['content']))

formated_slides

[Slides(id='SL_001', title="Biochimie du Gène et de l'Expression Génique", content="Biochimie du Gène et de l'Expression Génique\n(13 h cours – 11 chapitres)\nDrs. Pascal Dollé, Didier Devys, Philippe Kastner\n(Institut de Génétique et de Biologie Moléculaire et\nCellulaire [IGBMC], Parc d'innovation, Illkirch)\nDr. Jean Muller (Inserm U 1112, Génétique Médicale,\nCRBS, Faculté de Médecine)"),
 Slides(id='SL_002', title='1. Structure et propriétés des acides nucléiques', content="P.\nDollé\nD.\nDevys\nSommaire / Plan du cours (1)\n1. Structure et propriétés des acides nucléiques\n2. Réplication de l'ADN\n3. Réparation de l'ADN (anomalies et mécanismes de correction)\n4. Recombinaison de l’ADN (définition, généralités)\n5. Transcription de l'ADN (synthèse des ARN)\n6. Modifications (maturation) de l'ARN chez les eucaryotes\n7. Régulation de la transcription\n8. Code génétique\n9. Traduction (synthèse des protéines) P. Kastner\n10. Organisation et évolution des génomes eucaryotes animaux

In [30]:
import pdfplumber
from statistics import mean

pdf_path = "./volume/slides/cours_4_plan.pdf"

with pdfplumber.open(pdf_path) as pdf:
    slides_dict = {}
    all_text = []

    for page in pdf.pages:
        text = page.extract_text(layout=True)
        # Remove multiple consecutive newlines
        text = '\n'.join(line for line in text.splitlines() if line.strip())
        all_text.append(text)
    
    full_text = '\n'.join(all_text)
    
print(full_text)

                        Chapitre 4 - Réplication et réparation de l’ADN           
          I. STRUCTURE ET PROPRIÉTÉS DES ACIDES NUCLÉIQUES                        
          1. Les nucléotides                                                      
               1.1. Les bases azotées                                             
                      1.1.1. Les bases pyrimidiques                               
                      1.1.2. Les bases puriques                                   
                      1.1.3. Les bases modifiées dans l’ADN et l’ARN              
                      1.1.4. Autres dérivés : molécules d’intérêt biologique et médical
                      1.1.5. Propriétés des bases azotées                         
                      1.1.6. Transformation chimique des bases                    
               1.2. Les nucléosides                                               
                      1.2.1. Le pentose                                           

In [31]:
from models import Content, ContentSection, SectionSlideMapping
from typing import List, Dict, Optional

In [32]:
import tiktoken
encoder = tiktoken.get_encoding("cl100k_base")  # GPT-4 encoding
text_course_outline = str(full_text)
text_course_content = str(slides)
token_count, token_count_outline = len(encoder.encode(text_course_content)), len(encoder.encode(text_course_outline))
print(token_count, token_count_outline)

27099 1490


In [33]:
import dotenv
dotenv.load_dotenv()

import openai

client = openai.OpenAI()

system_prompt = """
You are a course structure analyzer specialized in French academic content.  
Your task is to parse a raw course outline text containing numbered section titles (like "I.", "1.", "1.1.", etc.) or indented bullet points.
and generate a hierarchical JSON structure matching this Pydantic model:

class ContentSection:
    id: str = ""
    title: str
    content: str = ""
    subsections: List[ContentSection]

class Content:
    sections: List[ContentSection]

Requirements:
- Detect all section titles and nest subsections correctly based on numbering.
- Maintain full recursive hierarchy.
- Return valid JSON exactly matching the Content model structure.
- Include every section and subsection with their titles.
- Ignore the content and the id of the sections, you only need to return the outline.
- Return ONLY the JSON, no additional text or explanation.
"""

user_message = full_text  # ton texte brut extrait

response = client.responses.parse(
    model="gpt-4.1-mini",
    input=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_message}
    ],
    text_format=Content,
    temperature=0
)

outline = response.output_parsed

print(outline.print_outline())

Course Outline:
Chapitre 4 - Réplication et réparation de l’ADN
  I. STRUCTURE ET PROPRIÉTÉS DES ACIDES NUCLÉIQUES
    1. Les nucléotides
      1.1. Les bases azotées
        1.1.1. Les bases pyrimidiques
        1.1.2. Les bases puriques
        1.1.3. Les bases modifiées dans l’ADN et l’ARN
        1.1.4. Autres dérivés : molécules d’intérêt biologique et médical
        1.1.5. Propriétés des bases azotées
        1.1.6. Transformation chimique des bases
      1.2. Les nucléosides
        1.2.1. Le pentose
        1.2.2. La liaison osidique
        1.2.3. Nomenclature
      1.3. Les nucléotides
        1.3.1. Nomenclature
        1.3.2. Nucléotides d’intérêt biologique
    2. Les acides nucléiques
      2.1. La liaison phosphodiester
      2.2. Structure spatiale des acides desoxyribonucléiques (ADN)
      2.3. Propriétés des ADN
        2.3.1. État ionique et solubilité
        2.3.2. Dénaturation de l’ADN / Effet sur l’absorbance
      2.4. Organisation des ADN
        2.4.1. Organ

In [34]:
while len(outline.sections) == 1:
    outline.sections = outline.sections[0].subsections

print(outline.sections[0])

id='' title='I. STRUCTURE ET PROPRIÉTÉS DES ACIDES NUCLÉIQUES' content=[] subsections=[ContentSection(id='', title='1. Les nucléotides', content=[], subsections=[ContentSection(id='', title='1.1. Les bases azotées', content=[], subsections=[ContentSection(id='', title='1.1.1. Les bases pyrimidiques', content=[], subsections=[]), ContentSection(id='', title='1.1.2. Les bases puriques', content=[], subsections=[]), ContentSection(id='', title='1.1.3. Les bases modifiées dans l’ADN et l’ARN', content=[], subsections=[]), ContentSection(id='', title='1.1.4. Autres dérivés : molécules d’intérêt biologique et médical', content=[], subsections=[]), ContentSection(id='', title='1.1.5. Propriétés des bases azotées', content=[], subsections=[]), ContentSection(id='', title='1.1.6. Transformation chimique des bases', content=[], subsections=[])]), ContentSection(id='', title='1.2. Les nucléosides', content=[], subsections=[ContentSection(id='', title='1.2.1. Le pentose', content=[], subsections=[

In [35]:
def assign_ids(sec: ContentSection, parent_id: str = "", parent_section: Optional[ContentSection] = None):
    # Determine current section number
    if parent_id == "":
        # Top level section
        sec.id = f"SEC_{len([s for s in outline.sections if s.id.startswith('SEC_')])+ 1}"
    else:
        # Subsection - append next number to parent id
        parent_nums = parent_id.replace("SEC_", "")
        sibling_count = len([s for s in parent_section.subsections if s.id.startswith(f"SEC_{parent_nums}.")])
        sec.id = f"SEC_{parent_nums}.{sibling_count + 1}"
    
    # Recursively assign ids to subsections
    for sub in sec.subsections:
        assign_ids(sub, sec.id, sec)

for s in outline.sections:
    assign_ids(s)
outline_json = outline.model_dump_json()


In [36]:
outline

Content(sections=[ContentSection(id='SEC_1', title='I. STRUCTURE ET PROPRIÉTÉS DES ACIDES NUCLÉIQUES', content=[], subsections=[ContentSection(id='SEC_1.1', title='1. Les nucléotides', content=[], subsections=[ContentSection(id='SEC_1.1.1', title='1.1. Les bases azotées', content=[], subsections=[ContentSection(id='SEC_1.1.1.1', title='1.1.1. Les bases pyrimidiques', content=[], subsections=[]), ContentSection(id='SEC_1.1.1.2', title='1.1.2. Les bases puriques', content=[], subsections=[]), ContentSection(id='SEC_1.1.1.3', title='1.1.3. Les bases modifiées dans l’ADN et l’ARN', content=[], subsections=[]), ContentSection(id='SEC_1.1.1.4', title='1.1.4. Autres dérivés : molécules d’intérêt biologique et médical', content=[], subsections=[]), ContentSection(id='SEC_1.1.1.5', title='1.1.5. Propriétés des bases azotées', content=[], subsections=[]), ContentSection(id='SEC_1.1.1.6', title='1.1.6. Transformation chimique des bases', content=[], subsections=[])]), ContentSection(id='SEC_1.1.2

In [37]:
print(outline.print_outline())

Course Outline:
I. STRUCTURE ET PROPRIÉTÉS DES ACIDES NUCLÉIQUES
  1. Les nucléotides
    1.1. Les bases azotées
      1.1.1. Les bases pyrimidiques
      1.1.2. Les bases puriques
      1.1.3. Les bases modifiées dans l’ADN et l’ARN
      1.1.4. Autres dérivés : molécules d’intérêt biologique et médical
      1.1.5. Propriétés des bases azotées
      1.1.6. Transformation chimique des bases
    1.2. Les nucléosides
      1.2.1. Le pentose
      1.2.2. La liaison osidique
      1.2.3. Nomenclature
    1.3. Les nucléotides
      1.3.1. Nomenclature
      1.3.2. Nucléotides d’intérêt biologique
  2. Les acides nucléiques
    2.1. La liaison phosphodiester
    2.2. Structure spatiale des acides desoxyribonucléiques (ADN)
    2.3. Propriétés des ADN
      2.3.1. État ionique et solubilité
      2.3.2. Dénaturation de l’ADN / Effet sur l’absorbance
    2.4. Organisation des ADN
      2.4.1. Organisation de l’ADN viral
      2.4.2. Organisation de l’ADN des bactéries
      2.4.3. Organisatio

## Pass 1.bis mapping slide number to section title

In [ ]:
def build_mapping_prompt(outline_json: str, slides: List[dict]) -> str:
    """
    Génère le prompt demandant au modèle de produire le mapping section→slides.

    Paramètres
    ----------
    outline_json : str   JSON du plan (chaque section possède un id SEC_***)
    slides       : list  Liste de dicts {"id": "SL_***", "title": ..., "content": ...}

    Retour
    ------
    str : prompt clair et contraignant, prêt pour client.responses.parse
    """
    return f"""
OBJECTIF
Associer chaque section du plan aux diapositives pertinentes.

DONNÉES
1) PLAN_JSON : (structure hiérarchique, chaque section a un identifiant unique "SEC_xxx")


This is the structure of the outline that you will receive:
class ContentSection(BaseModel):
    id: str
    title: str
    content: Optional[str] = ""
    subsections: List["ContentSection"] = Field(default_factory=list)
    
class Content(BaseModel):
    sections: List[ContentSection]

This is the actual outline:

{outline_json}

          


2) SLIDES : (liste ordonnée, chaque slide a un identifiant unique "SL_xxx")
{slides}

FORMAT DE SORTIE — STRICTEMENT OBLIGATOIRE
Un JSON **valide** contenant UNE SEULE clé de premier niveau : "mapping".

This is the pydantic model SectionSlideMapping that you will use to return the mapping:

class MappingItem(BaseModel):
    section_id: str
    slide_ids:  List[str]

class SectionSlideMapping(BaseModel):
    mapping: List[MappingItem]


Exemple de structure attendue :
{{
  "mapping": [
    {{ "section_id": "SEC_1", "slide_ids": ["SL_001", "SL_002"] }},
    {{ "section_id": "SEC_1.1", "slide_ids": ["SL_003", "SL_004"] }}
  ]
}}

CONTRAINTES À RESPECTER IMPÉRATIVEMENT
- Chaque "section_id" présent dans PLAN_JSON doit apparaître **exactement une fois**.
- Chaque "slide_id" présent dans SLIDES doit apparaître **au moins une fois** (aucune slide orpheline).
- "slide_ids" est un tableau d'identifiants ; l’ordre interne n’a pas d’importance.
- N’utilise **que** des identifiants existants ; n’invente pas de clés ni de champs.
- AUCUNE explication, commentaire ou clé supplémentaire : la sortie doit être **uniquement** le JSON demandé.
""".strip()


mapping_rsp = client.responses.parse(
    model="gpt-4.1-mini",
    input=[{"role":"user",
            "content": build_mapping_prompt(outline_json, slides)}],
    text_format=SectionSlideMapping,          # ← schéma fixe
    temperature=0
)
slides_by_section: Dict[str, List[str]] = {
    item.section_id: item.slide_ids
    for item in mapping_rsp.output_parsed.mapping
}


In [39]:
section_mapping = mapping_rsp.output_parsed

print(section_mapping.visualize_mapping(outline))

Section-to-Slides Mapping:

[Root] SEC_1
Title: I. STRUCTURE ET PROPRIÉTÉS DES ACIDES NUCLÉIQUES
Slides: 2 slide(s)
Slide IDs: SL_002, SL_006

  [Level 1] SEC_1.1
  Title: 1. Les nucléotides
  Slides: 1 slide(s)
  Slide IDs: SL_007

    [Level 2] SEC_1.1.1
    Title: 1.1. Les bases azotées
    Slides: 10 slide(s)
    Slide IDs: SL_008, SL_009, SL_010, SL_011, SL_012, SL_013, SL_014, SL_015, SL_016, SL_017

    [Level 2] SEC_1.1.2
    Title: 1.2. Les nucléosides
    Slides: 4 slide(s)
    Slide IDs: SL_018, SL_019, SL_020, SL_021

    [Level 2] SEC_1.1.3
    Title: 1.3. Les nucléotides
    Slides: 3 slide(s)
    Slide IDs: SL_022, SL_023, SL_024

  [Level 1] SEC_1.2
  Title: 2. Les acides nucléiques
  Slides: 1 slide(s)
  Slide IDs: SL_025

    [Level 2] SEC_1.2.1
    Title: 2.1. La liaison phosphodiester
    Slides: 2 slide(s)
    Slide IDs: SL_026, SL_027

    [Level 2] SEC_1.2.2
    Title: 2.2. Structure spatiale des acides desoxyribonucléiques (ADN)
    Slides: 8 slide(s)
    Slide 

In [13]:
from models import CourseWithSlides
course_with_slides = CourseWithSlides(content=outline, slides=formated_slides, mapping=section_mapping)
course_with_slides

CourseWithSlides(content=Content(sections=[ContentSection(id='SEC_1', title='Introduction', content=['L1SpS: UE 2 Les molécules du vivant\nPlan du cours\n• Introduction\n• Notions fondamentales\n• Le génome humain\n• Projet de séquençage\n• Architecture globale\n• Génome mitochondrial\n• Génome nucléaire\n• Description des principaux éléments constituants\n• Gènes (codants et non-codants), pseudogènes et éléments répétés\n• Comparaison aux autres génomes\n• Les types de variations du génome humain et leurs conséquences\n• La variabilité du génome humain\n• Evolution des génomes: notions essentielles, mécanismes\n2', 'L1SpS: UE 2 Les molécules du vivant\nIntroduction\n3', 'L1SpS: UE 2 Les molécules du vivant\nL’arbre de la vie\nNotions essentielles:\n• Evolution\n• Temps\n• Adaptation\n• Sélection\nLUCA\nLast Universal Common Ancestor\n4', 'L1SpS: UE 2 Les molécules du vivant\nArbre des eucaryotes\nChoix d’organismes modèles\n• Intérêt(s) économique(s)\n• Intérêt(s) technique(s)\n• Faci

In [14]:
new_content = course_with_slides.content
type(new_content)

models.Content

# Volumen of tokens

In [15]:
import tiktoken
encoder = tiktoken.get_encoding("cl100k_base")  # GPT-4 encoding
text_course_with_slides = str(course_with_slides)
token_count = len(encoder.encode(text_course_with_slides))
print(token_count)

26206


In [16]:
content_to_send = Content(sections=new_content.sections[:])
content_json_str = content_to_send.model_dump_json()
content_json_str

'{"sections":[{"id":"SEC_1","title":"Introduction","content":["L1SpS: UE 2 Les molécules du vivant\\nPlan du cours\\n• Introduction\\n• Notions fondamentales\\n• Le génome humain\\n• Projet de séquençage\\n• Architecture globale\\n• Génome mitochondrial\\n• Génome nucléaire\\n• Description des principaux éléments constituants\\n• Gènes (codants et non-codants), pseudogènes et éléments répétés\\n• Comparaison aux autres génomes\\n• Les types de variations du génome humain et leurs conséquences\\n• La variabilité du génome humain\\n• Evolution des génomes: notions essentielles, mécanismes\\n2","L1SpS: UE 2 Les molécules du vivant\\nIntroduction\\n3","L1SpS: UE 2 Les molécules du vivant\\nL’arbre de la vie\\nNotions essentielles:\\n• Evolution\\n• Temps\\n• Adaptation\\n• Sélection\\nLUCA\\nLast Universal Common Ancestor\\n4","L1SpS: UE 2 Les molécules du vivant\\nArbre des eucaryotes\\nChoix d’organismes modèles\\n• Intérêt(s) économique(s)\\n• Intérêt(s) technique(s)\\n• Facilités héber

In [17]:
content_to_send.sections[0].content.pop(0)

'L1SpS: UE 2 Les molécules du vivant\nPlan du cours\n• Introduction\n• Notions fondamentales\n• Le génome humain\n• Projet de séquençage\n• Architecture globale\n• Génome mitochondrial\n• Génome nucléaire\n• Description des principaux éléments constituants\n• Gènes (codants et non-codants), pseudogènes et éléments répétés\n• Comparaison aux autres génomes\n• Les types de variations du génome humain et leurs conséquences\n• La variabilité du génome humain\n• Evolution des génomes: notions essentielles, mécanismes\n2'

In [18]:
for i in content_to_send.sections[0].content:
    print ("="*100)
    print(i)
    print ("="*100)






L1SpS: UE 2 Les molécules du vivant
Introduction
3
L1SpS: UE 2 Les molécules du vivant
L’arbre de la vie
Notions essentielles:
• Evolution
• Temps
• Adaptation
• Sélection
LUCA
Last Universal Common Ancestor
4
L1SpS: UE 2 Les molécules du vivant
Arbre des eucaryotes
Choix d’organismes modèles
• Intérêt(s) économique(s)
• Intérêt(s) technique(s)
• Facilités hébergement,
• Développement embryonnaire/temps
de génération
• Partage d’éléments communs (ADN)
• Hérités/Transmis
35 eucaryotes
18 métazoaires
5
L1SpS: UE 2 Les molécules du vivant
A propos de l’homologie
• Définition
• En biologie, l'homologie désigne un caractère propre à plusieurs
espèces hérité d'un ancêtre commun. Les structures en question, qu'elles
soient d'ordre anatomique, moléculaire ou génétique, partagent donc une
histoire évolutive.
• ≠ analogie (ex: ailes oiseaux et insectes qui est une évolution séparée)
• Orthologie (gènes orthologues): gènes homologues dans 2 espèces ayant
évolué à partir d’un gène ancestral suite 

In [19]:
encoder = tiktoken.get_encoding("cl100k_base")  # GPT-4 encoding
text_course_with_slides = str(content_json_str)
token_count = len(encoder.encode(text_course_with_slides))
print(token_count)

12309


In [24]:
def build_system_prompt() -> str:
    """
    Prompt système définissant le rôle et les règles
    """
    return """Tu es un expert en rédaction pédagogique. Ta tâche est de transformer du contenu brut de slides en paragraphes de cours structurés.

Règles :
- Traite TOUTES les sections du JSON (ne t'arrête pas à la première)
- Pour chaque section, synthétise tous les éléments content[] en 2-10 paragraphes cohérents
- Chaque paragraphe = maximum 10 phrases liées au titre de la section
- Conserve la structure JSON exacte (mêmes IDs, titres, ordre)
- Style académique fluide"""


def build_user_prompt(content_json: str) -> str:
    """
    Prompt utilisateur avec le contenu à traiter
    """
    return content_json


def build_assistant_prompt() -> str:
    """
    Prompt assistant avec exemple d'input/output
    """
    return """Voici un exemple de transformation attendue :

INPUT EXEMPLE:
```json
{
  "sections": [
    {
      "id": "SEC_1",
      "title": "Introduction aux Algorithmes",
      "content": [
        "CS101: Algorithmique\\nAlgorithmes\\n• Définition: séquence d'instructions\\n• Résolution problèmes\\n• Entrée → Traitement → Sortie\\n• Propriétés essentielles\\n1",
        "CS101: Algorithmique\\nCaractéristiques\\n• Fini (nombre étapes)\\n• Non-ambigu\\n• Déterministe\\n• Efficacité importante\\n• Complexité temporelle/spatiale\\n2",
        "CS101: Algorithmique\\nTypes d'algorithmes\\n• Itératifs vs Récursifs\\n• Diviser pour régner\\n• Gloutons (greedy)\\n• Programmation dynamique\\nExemples concrets\\n3"
      ],
      "subsections": [
        {
          "id": "SEC_1.1",
          "title": "Notation Big-O",
          "content": [
            "CS101: Algorithmique\\nComplexité algorithmique\\n• Notation O(n)\\n• Mesure efficacité\\n• Pire cas considéré\\n• Croissance asymptotique\\n4",
            "CS101: Algorithmique\\nExemples complexités\\n• O(1) - constant\\n• O(log n) - logarithmique\\n• O(n) - linéaire\\n• O(n²) - quadratique\\n• O(2^n) - exponentielle\\n5"
          ],
          "subsections": []
        }
      ]
    },
    {
      "id": "SEC_2", 
      "title": "Structures de Données",
      "content": [
        "CS101: Algorithmique\\nStructures fondamentales\\n• Tableaux (arrays)\\n• Listes chaînées\\n• Piles (stacks)\\n• Files (queues)\\n• Organisation mémoire\\n6",
        "CS101: Algorithmique\\nOpérations de base\\n• Insertion / Suppression\\n• Recherche / Accès\\n• Parcours / Tri\\n• Complexité variable\\n• Trade-offs temps/espace\\n7"
      ],
      "subsections": []
    }
  ]
}
```

OUTPUT EXEMPLE:
```json
{
  "sections": [
    {
      "id": "SEC_1",
      "title": "Introduction aux Algorithmes", 
      "content": [
        "Un algorithme constitue une séquence finie et ordonnée d'instructions permettant de résoudre un problème spécifique ou d'accomplir une tâche donnée. Cette approche systématique transforme des données d'entrée en résultats de sortie par l'application d'un processus de traitement déterministe et reproductible.",
        "Les algorithmes présentent des caractéristiques fondamentales qui garantissent leur validité : ils doivent être finis dans le nombre d'étapes, non-ambigus dans leur interprétation, et déterministes dans leur exécution. L'efficacité algorithmique, mesurée par les complexités temporelle et spatiale, constitue un critère essentiel d'évaluation de leur performance.",
        "La classification des algorithmes distingue plusieurs approches méthodologiques selon leur stratégie de résolution : les algorithmes itératifs et récursifs, les méthodes diviser-pour-régner, les algorithmes gloutons, et la programmation dynamique. Chaque paradigme offre des avantages spécifiques selon la nature du problème à traiter."
      ],
      "subsections": [
        {
          "id": "SEC_1.1",
          "title": "Notation Big-O",
          "content": [
            "La notation Big-O fournit un outil mathématique pour caractériser la complexité algorithmique en décrivant la croissance asymptotique du temps d'exécution ou de l'espace mémoire requis. Cette mesure considère le comportement dans le pire cas et permet de comparer objectivement l'efficacité de différents algorithmes.",
            "Les classes de complexité courantes s'organisent selon une hiérarchie croissante : O(1) pour les opérations à temps constant, O(log n) pour les algorithmes logarithmiques, O(n) pour les traitements linéaires, O(n²) pour les approches quadratiques, et O(2^n) pour les solutions exponentielles. Cette classification guide le choix algorithmique selon les contraintes de performance requises."
          ],
          "subsections": []
        }
      ]
    },
    {
      "id": "SEC_2",
      "title": "Structures de Données",
      "content": [
        "Les structures de données fondamentales organisent et stockent l'information selon des patterns spécifiques adaptés aux besoins algorithmiques : les tableaux offrent un accès direct par index, les listes chaînées permettent une allocation dynamique, tandis que les piles et files implémentent des politiques d'accès particulières. L'organisation mémoire influence directement les performances d'accès et de manipulation.",
        "Les opérations de base sur ces structures - insertion, suppression, recherche, accès, parcours et tri - présentent des complexités variables selon l'implémentation choisie. Ces trade-offs entre temps d'exécution et espace mémoire orientent la sélection de la structure la plus appropriée selon les contraintes spécifiques de chaque application."
      ],
      "subsections": []
    }
  ]
}
```

Maintenant traite le JSON fourni en suivant ce modèle :"""


def build_prompt_fill_content(
    content_json: str
) -> str:
    """
    Version legacy - maintenant utilise les 3 fonctions séparées ci-dessus
    """
    return f"""
**SYSTEM:**
{build_system_prompt()}

**USER:**
{build_user_prompt(content_json)}

**ASSISTANT:**
{build_assistant_prompt()}
""".strip()


prompt_to_send = build_prompt_fill_content(content_json_str)

response = client.responses.parse(
    model="gpt-4.1-mini",
    input=[
        {"role": "system", "content": "Tu es un Professeur qui remplit le contenu des sections du plan de cours à partir du contenu des slides."},
        {"role": "user", "content": prompt_to_send}
    ],
    text_format=Content,
    temperature=0,
)

print(response.output_parsed)

sections=[ContentSection(id='SEC_1', title='Introduction', content=["Ce cours intitulé « Les molécules du vivant » aborde une large gamme de sujets fondamentaux liés au génome humain et à son évolution. Le plan du cours inclut une introduction aux notions fondamentales, une étude détaillée du génome humain, le projet de séquençage, ainsi que l'architecture globale du génome. Il traite également des génomes mitochondrial et nucléaire, des principaux éléments constituants, des gènes codants et non-codants, des pseudogènes et des éléments répétés. Une comparaison avec d'autres génomes est proposée, ainsi qu'une analyse des types de variations du génome humain et de leurs conséquences. Enfin, le cours explore la variabilité du génome humain et les mécanismes essentiels de son évolution.", "L'arbre de la vie est présenté comme un concept central, illustrant les notions d'évolution, de temps, d'adaptation et de sélection. Le dernier ancêtre commun universel (LUCA) est évoqué comme point de d

In [25]:
course = response.output_parsed
print(course.print_outline())

Course Outline:
Introduction
  Notions fondamentales
Le génome humain
  Projet de séquençage
  Architecture globale
    Génome mitochondrial
    Génome nucléaire
  Description des principaux éléments constituants
    Gènes (codants et non-codants), pseudogènes et éléments répétés
Comparaison aux autres génomes
Les types de variations du génome humain et leurs conséquences
La variabilité du génome humain
Evolution des génomes: notions essentielles, mécanismes


In [26]:
print(course.print_content())

Course Content:

[Root] ID: SEC_1
Title: Introduction
Content:
  [1] Ce cours intitulé « Les molécules du vivant » aborde une large gamme de
      sujets fondamentaux liés au génome humain et à son évolution. Le plan du
      cours inclut une introduction aux notions fondamentales, une étude
      détaillée du génome humain, le projet de séquençage, ainsi que
      l'architecture globale du génome. Il traite également des génomes
      mitochondrial et nucléaire, des principaux éléments constituants, des
      gènes codants et non-codants, des pseudogènes et des éléments répétés. Une
      comparaison avec d'autres génomes est proposée, ainsi qu'une analyse des
      types de variations du génome humain et de leurs conséquences. Enfin, le
      cours explore la variabilité du génome humain et les mécanismes essentiels
      de son évolution.
  [2] L'arbre de la vie est présenté comme un concept central, illustrant les
      notions d'évolution, de temps, d'adaptation et de sélection. L

In [23]:
encoder = tiktoken.get_encoding("cl100k_base")  # GPT-4 encoding
text_course_with_slides = str(course)
token_count = len(encoder.encode(text_course_with_slides))
print(token_count)

3992
